In [ ]:
!pip install weaviate-client -U

## Query data

In [50]:
import os
import weaviate

auth_config = weaviate.AuthBearerToken(
  access_token = os.getenv("COHERE_AUTH_READ"),
  expires_in=300, # in seconds, by default 60s
)

client = weaviate.Client(
    url="https://cohere-demo.weaviate.network",
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"),
    },
    timeout_config = (20, 240),
)
client.is_ready()

True

In [7]:
def semantic_serch(query):
    nearText = {
        "concepts": [query]
    }

    properties = [
        "text", "title", "url", "views",
        "_additional {distance}"
    ]

    response = (
        client.query
        .get("Articles", properties)
        .with_near_text(nearText)
        .with_limit(20)
        .do()
    )

    return response['data']['Get']['Articles']


def print_result(result):
    for item in result:
        print(f"\033[95m{item['title']} ({item['views']}) {item['_additional']['distance']}\033[0m")
        print(f"\033[4m{item['url']}\033[0m")
        print(item['text'])
        print()

In [62]:
# Pass the user query to Weaviate
query_result = semantic_serch("instrumenty muzyczne")

# Print out the result
print_result(query_result)

Instrument de musique (800) -151.23499
https://fr.wikipedia.org/wiki?curid=1521
L'ensemble des instruments utilisés pour une œuvre mais aussi, et surtout, dans une société donnée ou une époque est appelé « instrumentarium ». L'étude académique des instruments de musique est appelée organologie et prend le plus souvent ses sources dans l'ethnomusicologie.

Instrument de musique (800) -151.05759
https://fr.wikipedia.org/wiki?curid=1521
Un instrument de musique est un objet pouvant produire un son contrôlé par un musicien — que cet objet soit conçu dans cet objectif, ou bien qu'il soit modifié ou écarté de son usage premier. La voix ou les mains, même si elles ne sont pas des objets à proprement parler, sont considérées comme des instruments de musique dès lors qu'elles participent à une œuvre musicale.

Instrumento musical (1000) -150.76285
https://es.wikipedia.org/wiki?curid=1489
Un instrumento musical es un objeto compuesto por la combinación de uno o más sistemas resonantes y medios p

In [43]:
def generative_serch(query):

    properties = [
        "text", "title", "url", "views",
        "_additional {distance}"
    ]

    response = (
        client.query
        .get('Articles', properties)
        .with_near_text({'concepts': [query]})
        .with_limit(4)
        .with_generate(single_prompt='summarize the following text in Dutch: {text}')
        .do()
    )

    return response['data']['Get']['Wikipedia']


def print_result(result):
    for item in result:
        print(f"\033[95m{item['title']} ({item['views']}) {item['_additional']['distance']}\033[0m")
        print(f"\033[4m{item['_additional']['generate']['singleResult']}\033[0m")
        print(f"\033[4m{item['url']}\033[0m")
        print(item['text'])
        print()

In [44]:
# Pass the user query to Weaviate
query_result = generative_serch("instrumenty muzyczne")

# Print out the result
print_result(query_result)

Entertainment (3000) -148.43791
In de muzikale entertainment worden instrumenten gebruikt die ofwel alleen de menselijke stem zijn of alleen instrumentaal zijn, of een combinatie van beide. De uitvoerders kunnen solisten zijn of deel uitmaken van een kleine of grote groep die een individueel, passerend, klein of groot publiek vermaakt. Zingen wordt over het algemeen begeleid door instrumenten, hoewel sommige vormen, zoals a capella en boventoonzang, onbegeleid zijn. Moderne concerten maken vaak gebruik van verschillende speciale effecten en theatrics om optredens van zang en dans te begeleiden.
https://en.wikipedia.org/wiki?curid=9262
The instruments used in musical entertainment are either solely the human voice (2, 6) or solely instrumental (1, 3) or some combination of the two (4, 5, 7, 8). Whether the performance is given by vocalists or instrumentalists, the performers may be soloists or part of a small or large group, in turn entertaining an audience that might be individual (10)

In [ ]:
# Test that all data has loaded – get object count
result = (
    client.query.aggregate("Articles")
    .with_near_vector()
    # .with_where({
    #     "path": ["lang_id"],
    #     "operator": "Equal",
    #     "valueInt": 6
    # })
    .with_meta_count()
    .do()
)
print("Object count: ", result["data"]["Aggregate"]["Wikipedia"])